In [660]:
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True
)

test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download = True
)

In [661]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [662]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [663]:
train_data.data.shape

torch.Size([60000, 28, 28])

In [664]:
test_data.data.shape

torch.Size([10000, 28, 28])

In [665]:
from torch.utils.data import DataLoader

loaders = {
    'train': DataLoader(train_data, batch_size=100, shuffle=True, num_workers=1),
    'test': DataLoader(test_data, batch_size=100, shuffle=True, num_workers=1)
}

In [666]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5, stride=1, padding=2)
        self.conv3 = nn.Conv2d(in_channels=20, out_channels=40, kernel_size=2, stride=1)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(in_features=40*3*3, out_features=50)
        self.fc2 = nn.Linear(in_features=50, out_features=10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = x.view(-1, 40*3*3)

        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return F.softmax(x, dim=1)



In [667]:
model1 = CNN()
model1.parameters()

<generator object Module.parameters at 0x314c01700>

In [668]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model1.parameters(), lr=0.0005)

In [669]:
epochs = 1000

def train(epoch):
    model1.train()

    for batch_idx, (data, target) in enumerate(loaders['train']):
        optimizer.zero_grad()
        
        output = model1(data)

        loss = loss_fn(output, target)
        loss.backward()
        
        optimizer.step()

        if batch_idx % 20 == 0:
            print(f"Epoch: {epoch} | [{batch_idx * len(data)} / {len(loaders['train'].dataset)}] | Loss: {loss}")

In [670]:
def test():
    model1.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in loaders['test']:
            output = model1(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(loaders['test'].dataset)
    print(f"Average Loss: {test_loss} | Correct: {correct} / 10000")

In [671]:
for epoch in range(1, 20):
    train(epoch)
    test()

Epoch: 1 | [0 / 60000] | Loss: 2.3023788928985596
Epoch: 1 | [2000 / 60000] | Loss: 2.2998993396759033
Epoch: 1 | [4000 / 60000] | Loss: 2.282865285873413
Epoch: 1 | [6000 / 60000] | Loss: 2.142268419265747
Epoch: 1 | [8000 / 60000] | Loss: 1.928012728691101
Epoch: 1 | [10000 / 60000] | Loss: 1.8643336296081543
Epoch: 1 | [12000 / 60000] | Loss: 1.8102039098739624
Epoch: 1 | [14000 / 60000] | Loss: 1.8738456964492798
Epoch: 1 | [16000 / 60000] | Loss: 1.8091446161270142
Epoch: 1 | [18000 / 60000] | Loss: 1.7465687990188599
Epoch: 1 | [20000 / 60000] | Loss: 1.6468725204467773
Epoch: 1 | [22000 / 60000] | Loss: 1.6792597770690918
Epoch: 1 | [24000 / 60000] | Loss: 1.6316406726837158
Epoch: 1 | [26000 / 60000] | Loss: 1.6293469667434692
Epoch: 1 | [28000 / 60000] | Loss: 1.5909587144851685
Epoch: 1 | [30000 / 60000] | Loss: 1.6005722284317017
Epoch: 1 | [32000 / 60000] | Loss: 1.6138843297958374
Epoch: 1 | [34000 / 60000] | Loss: 1.605738639831543
Epoch: 1 | [36000 / 60000] | Loss: 1.624

In [672]:
from pathlib import Path

MODEL_PATH = Path("Models")
MODEL_PATH.mkdir(parents=1, exist_ok=1)

MODEL_NAME = "01_model_0.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model1.state_dict(), f=MODEL_SAVE_PATH)

Saving model to: Models/01_model_0.pth
